# A notebook to compile counts for the HRApop paper

# Import libraries

In [1]:
%pip install pandas numpy requests

import pandas as pd
import numpy as np
import requests
import io


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


# Set global variables

In [2]:
hra_pop_version = "v0.12.0"
branch = 'v0.12.0'

# Load data

In [3]:
sankey = pd.read_csv(
    f"https://raw.githubusercontent.com/x-atlas-consortia/hra-pop/refs/heads/{branch}/output-data/{hra_pop_version}/reports/universe-ad-hoc/sankey.csv")

sankey

,portal,study_paper,doi,lead_author,is_azimuth_reference,donor_id,donor_sex,donor_age,donor_development_stage,donor_race,...,unique_dataset_id,link_to_h5ad_file,sc_transcriptomics_or_sc_proteomics,cell_type_annotation_tool,omap_id,number_of_cells_total,number_of_unique_cell_types,hubmap_dataset_publication_status,is_rui_registered,is_atlas_dataset
0,KPMP,NaN,http://dx.doi.org/10.1681/ASN.2016091027,NaN,NaN,Donor1,Male,NaN,NaN,NaN,...,http://dx.doi.org/10.1681/ASN.2016091027#Donor...,http://dx.doi.org/10.1681/ASN.2016091027,NaN,NaN,NaN,NaN,NaN,NaN,True,False
1,KPMP,NaN,http://dx.doi.org/10.1681/ASN.2016091027,NaN,NaN,Donor2,Male,NaN,NaN,NaN,...,http://dx.doi.org/10.1681/ASN.2016091027#Donor...,http://dx.doi.org/10.1681/ASN.2016091027,NaN,NaN,NaN,NaN,NaN,NaN,True,False
2,KPMP,NaN,http://dx.doi.org/10.1681/ASN.2016091027,NaN,NaN,Donor3,Male,NaN,NaN,NaN,...,http://dx.doi.org/10.1681/ASN.2016091027#Donor...,http://dx.doi.org/10.1016/j.trsl.2017.07.006,NaN,NaN,NaN,NaN,NaN,NaN,True,False
3,HRA,NaN,NaN,NaN,NaN,Donor1,Female,38.0,NaN,NaN,...,http://purl.org/ccf/1.5/omap-1#Donor1_TissueBl...,https://hubmapconsortium.github.io/ccf-release...,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4,"HIRN, ESPACE",NaN,https://doi.org/10.5281/zenodo.7742474,NaN,NaN,Donor1,Male,NaN,NaN,NaN,...,http://purl.org/ccf/1.5/omap-6#Donor1_TissueBl...,https://zenodo.org/record/7742474,NaN,NaN,NaN,NaN,NaN,NaN,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22459,SPARC,NaN,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,Donor1,Female,49.0,NaN,NaN,...,https://sparc.science/datasets/390/version/1?d...,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,NaN,NaN,NaN,NaN,True,False
22460,SPARC,NaN,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,Donor1,Female,49.0,NaN,NaN,...,https://sparc.science/datasets/390/version/1?d...,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,NaN,NaN,NaN,NaN,True,False
22461,SPARC,NaN,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,Donor1,Female,49.0,NaN,NaN,...,https://sparc.science/datasets/390/version/1?d...,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,NaN,NaN,NaN,NaN,True,False
22462,KPMP,NaN,https://www.nature.com/articles/s41467-023-389...,NaN,NaN,Donor1,Male,NaN,NaN,NaN,...,https://zenodo.org/records/7653239#Donor1_Tiss...,https://zenodo.org/records/7653239,NaN,NaN,NaN,NaN,NaN,NaN,True,False


In [4]:
# unique cells
universe_sc_transcriptomics_cell_counts = pd.read_csv(
    f'https://raw.githubusercontent.com/x-atlas-consortia/hra-pop/refs/heads/{branch}/output-data/{hra_pop_version}/reports/universe-ad-hoc/universe-sc-transcriptomics-cell-counts.csv', index_col=False)
universe_sc_proteomics_cell_counts = pd.read_csv(
    f'https://raw.githubusercontent.com/x-atlas-consortia/hra-pop/refs/heads/{branch}/output-data/{hra_pop_version}/reports/universe-ad-hoc/universe-sc-proteomics-cell-counts.csv', index_col=False)
universe_sc_transcriptomics_cell_instance_counts = pd.read_csv(
    f'https://raw.githubusercontent.com/x-atlas-consortia/hra-pop/refs/heads/{branch}/output-data/{hra_pop_version}/reports/universe-ad-hoc/universe-sc-transcriptomics-cell-instance-counts.csv', index_col=False)

# Pre-processing steps

## Simplify rows with multiple annotations for getting accurate counts

In [5]:
tool_replacement = "sc_transcriptomics with Cell Summary"

sankey['cell_type_annotation_tool'] = sankey['cell_type_annotation_tool'].replace({
    'azimuth': tool_replacement,
    'celltypist': tool_replacement,
    'popv': tool_replacement,
    np.nan: "No Cell Summary"
})

## Manually fix missing cell type annotation values for SenNet atlas datasets

See GitHub issue: https://github.com/x-atlas-consortia/hra-pop/issues/91

In [6]:
# Define the indexing criteria
criteria = (sankey['portal'] == "SenNet") & (
    sankey['is_atlas_dataset'] == True)

# Apply the change to the SenNet atlas datasets (2 as of HRApop v0.10.2)
sankey.loc[criteria, 'cell_type_annotation_tool'] = tool_replacement

sankey = sankey.drop_duplicates()

# Get counts for HRApop paper

The following sections provide counts of datasets and other metrics for HRApop v0.10.2.

## Report numbers for Highlights

In [7]:
# All datasets downloaded and retrieved from extraction sites
all_datasets = sankey['unique_dataset_id'].unique()

print(f"Number of UNIVERSE datasets: {len(all_datasets)}")

Number of UNIVERSE datasets: 15813


In [8]:
# All sc-proteomics
all_sc_proteomics = sankey[['dataset_id', 'cell_type_annotation_tool']
                           ].loc[sankey['cell_type_annotation_tool'] == "sc_proteomics"].drop_duplicates()

print(f"Number of sc-proteomics datasets: {len(all_sc_proteomics)}")

Number of sc-proteomics datasets: 107


In [9]:
print(
    f'Number of sc-transcriptomics datasets: {len(sankey[(sankey['is_atlas_dataset'] == True) & (sankey['cell_type_annotation_tool'] != 'sc_proteomics')].drop_duplicates(subset=['unique_dataset_id']))}')

Number of sc-transcriptomics datasets: 607


In [10]:
# All datasets with cell summary
sc_transcriptomics_with_cell_summary = sankey[['unique_dataset_id', 'cell_type_annotation_tool']].loc[
    sankey['cell_type_annotation_tool'] == tool_replacement]['unique_dataset_id'].drop_duplicates()

print(f"Number of sc-transcriptomics datasets with cell summary: {len(sc_transcriptomics_with_cell_summary)}")

Number of sc-transcriptomics datasets with cell summary: 7107


In [11]:
# Organs in HRApop Atlas
organs_in_hra_pop = sankey.loc[sankey['is_atlas_dataset']
                                == True]['organ_name'].unique()
print(
    f"Unique organs in HRApop Atlas: {len(organs_in_hra_pop)}")

Unique organs in HRApop Atlas: 17


In [12]:
# Organs (m/f) in HRApop Atlas
organs_in_hra_pop_sex = sankey.loc[sankey['is_atlas_dataset']
                               == True]['organ_name_glb_file'].unique()
print(
    f"Organs (m/f) in HRApop Atlas: {len(organs_in_hra_pop_sex)}")

Organs (m/f) in HRApop Atlas: 31


In [13]:
# Volume covered by HRApop tissue blocks
volume = sankey.loc[sankey['is_atlas_dataset']== True].drop_duplicates(subset=['unique_dataset_id'])['tissue_block_volume'].sum()
print(
    f"Volume covered by HRApop tissue blocks: {volume}")

Volume covered by HRApop tissue blocks: 5659134.858


## Report numbers for Sankey/experimental data

In [14]:
# atlas datasets
atlas = sankey.loc[sankey['is_atlas_dataset'] == True]['unique_dataset_id'].unique()
print(f"Atlas datasets: {len(atlas)}\n")

# datasets with extraction site but without cell summary
no_cell_summary = sankey.loc[(sankey['is_rui_registered'] == True) & (
    sankey['cell_type_annotation_tool'] == "No Cell Summary")]['unique_dataset_id'].unique()
print(f"Datasets with extraction site but without cell summary: {
      len(no_cell_summary)}\n")

# datasets with cell summary but without extraction site
no_rui = sankey.loc[(sankey['is_rui_registered'] ==False) & (
    sankey['cell_type_annotation_tool'] != "No Cell Summary")]
print(f"Datasets with cell summary but without extraction site: {
      len(no_rui)}\n")

# datasets with cell summary 
cell_summary = sankey.loc[sankey['cell_type_annotation_tool'] != "No Cell Summary"]
print(f"Datasets with cell summary: {
      len(cell_summary)}\n")

# datasets with neither
non_atlas_without_either = sankey.loc[(sankey['cell_type_annotation_tool'] == "No Cell Summary") & (sankey['is_rui_registered'] == False)]
print(f"Datasets with neither: {len(non_atlas_without_either)}\n")

# non-atlas datasets total
non_atlas_total = sankey.loc[(
    sankey['is_atlas_dataset'] == False)]['unique_dataset_id'].unique()
print(f"Non-atlas datasets total: {len(non_atlas_total)}\n")

# unique cells
sc_transcriptomics_cell_counts = universe_sc_transcriptomics_cell_counts[
    'universe_sc_transcriptomics_cell_count'].iloc[0]
print(
    f'Unique cells from sc-transcriptomics datasets in the Universe: {sc_transcriptomics_cell_counts}\n')

sc_transcriptomics_cell_counts_preannotated = universe_sc_transcriptomics_cell_counts[
    'universe_sc_transcriptomics_preannotated_cell_count'].iloc[0]
print(
    f'Unique cells from sc-transcriptomics datasets in the Universe (preannotated): {sc_transcriptomics_cell_counts_preannotated}\n')

sc_proteomics_cell_counts = universe_sc_proteomics_cell_counts[
    'universe_sc_proteomics_cell_count'].iloc[0]
print(
    f'Unique cells from sc-proteomics datasets in the Universe: {sc_proteomics_cell_counts}\n')

Atlas datasets: 714

Datasets with extraction site but without cell summary: 5208

Datasets with cell summary but without extraction site: 6500

Datasets with cell summary: 7214

Datasets with neither: 3391

Non-atlas datasets total: 15099

Unique cells from sc-transcriptomics datasets in the Universe: 40364420

Unique cells from sc-transcriptomics datasets in the Universe (preannotated): 40434437

Unique cells from sc-proteomics datasets in the Universe: 17547511



In [15]:
# h5ad files
print(f'Unique h5ad files: {sankey['link_to_h5ad_file'].nunique()}')

Unique h5ad files: 5755


## AS Counts

In [16]:
# Read the CSV data
df_as_data = pd.read_csv(
    f'https://raw.githubusercontent.com/x-atlas-consortia/hra-pop/refs/heads/{branch}/output-data/{hra_pop_version}/reports/atlas-ad-hoc/cell-types-in-anatomical-structurescts-per-as.csv')

# Display the DataFrame
df_as_data

,organ,as,as_label,sex,tool,modality,cell_id,cell_label,cell_count,cell_percentage,dataset_count
0,pancreas,http://purl.obolibrary.org/obo/UBERON_0001069,head of pancreas,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0002079,ductal,15.312,0.522523,1
1,pancreas,http://purl.obolibrary.org/obo/UBERON_0001069,head of pancreas,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0002064,acinar,8.640,0.294840,1
2,pancreas,http://purl.obolibrary.org/obo/UBERON_0001069,head of pancreas,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0000115,endothelial,3.864,0.131859,1
3,pancreas,http://purl.obolibrary.org/obo/UBERON_0001069,head of pancreas,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0000738,immune,1.464,0.049959,1
4,pancreas,http://purl.obolibrary.org/obo/UBERON_0001069,head of pancreas,Female,azimuth,sc_transcriptomics,http://purl.obolibrary.org/obo/CL_0002410,activated_stellate,0.024,0.000819,1
...,...,...,...,...,...,...,...,...,...,...,...
9216,lung,http://purl.org/sig/ont/fma/fma7508,Left posterior basal segmental bronchus,Male,sc_proteomics,sc_proteomics,http://purl.obolibrary.org/obo/CL_0000097,Mast Cell,15322.464,0.024702,1
9217,lung,http://purl.org/sig/ont/fma/fma7508,Left posterior basal segmental bronchus,Male,sc_proteomics,sc_proteomics,http://purl.obolibrary.org/obo/CL_4033039,CD8+ T Cell,3691.176,0.005951,1
9218,lung,http://purl.org/sig/ont/fma/fma7508,Left posterior basal segmental bronchus,Male,sc_proteomics,sc_proteomics,https://purl.org/ccf/ASCTB-TEMP_lymphatic-endo...,Lymphatic Endothelial (and some immune cells),1753.956,0.002828,1
9219,lung,http://purl.org/sig/ont/fma/fma7508,Left posterior basal segmental bronchus,Male,sc_proteomics,sc_proteomics,https://purl.org/ccf/ASCTB-TEMP_basal-epitheli...,Basal Epithelial Cell,970.104,0.001564,1


In [17]:
# unique AS
print(f'Number of unique AS IDs in HRApop {hra_pop_version}: {len(df_as_data['as_label'].unique())}')

Number of unique AS IDs in HRApop v0.12.0: 74


In [18]:
unique_as_by_sex = df_as_data[['as_label', 'sex']].drop_duplicates()
print(
    f'Number of unique AS in HRApop {hra_pop_version} separated by sex: {len(unique_as_by_sex)}')

Number of unique AS in HRApop v0.12.0 separated by sex: 115


In [19]:
print(f'Number of organs covered by sc-transcriptomics: {sankey[(sankey['is_atlas_dataset'] == True) & (sankey['cell_type_annotation_tool'] != 'sc_proteomics')]['organ_name'].nunique()}')

Number of organs covered by sc-transcriptomics: 17


In [27]:
print(f'Number of AS covered by sc-proteomics: {df_as_data[df_as_data['tool'] == 'sc_proteomics']['as_label'].nunique()}')

Number of AS covered by sc-proteomics: 17


# Random queries

In [21]:
# get ATLAS datasets with donors < 18
underage = sankey[(sankey['donor_age'] < 18) & (sankey['is_atlas_dataset'] == True)]
underage

,portal,study_paper,doi,lead_author,is_azimuth_reference,donor_id,donor_sex,donor_age,donor_development_stage,donor_race,...,unique_dataset_id,link_to_h5ad_file,sc_transcriptomics_or_sc_proteomics,cell_type_annotation_tool,omap_id,number_of_cells_total,number_of_unique_cell_types,hubmap_dataset_publication_status,is_rui_registered,is_atlas_dataset
4243,NHLBI/LungMap,LungMAP ��� Human data from a broad age health...,https://doi.org/10.7554/eLife.62522,Allen Wang,NaN,D032_Donor,Male,3.00,NaN,NaN,...,https://api.cellxgene.cziscience.com/dp/v1/col...,https://data-browser.lungmap.net/explore/proje...,NaN,sc_transcriptomics with Cell Summary,NaN,14910.0,70.0,NaN,True,True
4246,NHLBI/LungMap,LungMAP ��� Human data from a broad age health...,https://doi.org/10.7554/eLife.62522,Allen Wang,NaN,D046_Donor,Male,3.00,NaN,NaN,...,https://api.cellxgene.cziscience.com/dp/v1/col...,https://data-browser.lungmap.net/explore/proje...,NaN,sc_transcriptomics with Cell Summary,NaN,26682.0,77.0,NaN,True,True
4259,NHLBI/LungMap,LungMAP ��� Human data from a broad age health...,https://doi.org/10.7554/eLife.62522,Allen Wang,NaN,D139_Donor,Female,3.00,NaN,NaN,...,https://api.cellxgene.cziscience.com/dp/v1/col...,https://data-browser.lungmap.net/explore/proje...,NaN,sc_transcriptomics with Cell Summary,NaN,15651.0,70.0,NaN,True,True
16829,HuBMAP,NaN,NaN,NaN,NaN,NaN,Female,14.00,NaN,White,...,https://entity.api.hubmapconsortium.org/entiti...,https://portal.hubmapconsortium.org/browse/dat...,NaN,sc_transcriptomics with Cell Summary,NaN,3650.0,34.0,NaN,True,True
16974,HuBMAP,NaN,NaN,NaN,NaN,NaN,Female,14.00,NaN,White,...,https://entity.api.hubmapconsortium.org/entiti...,https://portal.hubmapconsortium.org/browse/dat...,NaN,sc_transcriptomics with Cell Summary,NaN,3650.0,34.0,NaN,True,True
17414,HuBMAP,NaN,NaN,NaN,NaN,NaN,Male,10.00,NaN,White,...,https://entity.api.hubmapconsortium.org/entiti...,https://portal.hubmapconsortium.org/browse/dat...,NaN,sc_transcriptomics with Cell Summary,NaN,2690.0,18.0,NaN,True,True
17436,HuBMAP,NaN,NaN,NaN,NaN,NaN,Male,10.00,NaN,White,...,https://entity.api.hubmapconsortium.org/entiti...,https://portal.hubmapconsortium.org/browse/dat...,NaN,sc_proteomics,NaN,226384.0,11.0,NaN,True,True
17737,HuBMAP,NaN,NaN,NaN,NaN,NaN,Male,10.00,NaN,White,...,https://entity.api.hubmapconsortium.org/entiti...,https://portal.hubmapconsortium.org/browse/dat...,NaN,sc_proteomics,NaN,130584.0,11.0,NaN,True,True
18541,HuBMAP,NaN,NaN,NaN,NaN,NaN,Female,17.00,NaN,White,...,https://entity.api.hubmapconsortium.org/entiti...,https://portal.hubmapconsortium.org/browse/dat...,NaN,sc_transcriptomics with Cell Summary,NaN,5359.0,13.0,NaN,True,True
19229,HuBMAP,NaN,NaN,NaN,NaN,NaN,Male,10.00,NaN,White,...,https://entity.api.hubmapconsortium.org/entiti...,https://portal.hubmapconsortium.org/browse/dat...,NaN,sc_transcriptomics with Cell Summary,NaN,13858.0,21.0,NaN,True,True
